In [0]:
%pip install yfinance

Downloading and Previewing yfinance data table for $NVDA

In [0]:
import yfinance as yf
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

ticker = "NVDA"
end = datetime.today()
start = end - timedelta(days=365 * 10)  # ~10 years

data = yf.download(ticker, start=start, end=end)
data.head()


Create a new Unity Catalogue Volume to store the data

In [0]:
# Use Unity Catalog's default catalog "main"
spark.sql("CREATE CATALOG IF NOT EXISTS main")  # might already exist, that's fine
spark.sql("CREATE SCHEMA IF NOT EXISTS main.stocks")
spark.sql("USE CATALOG main")
spark.sql("USE SCHEMA stocks")

# Create a managed volume to hold the raw CSV files
spark.sql("CREATE VOLUME IF NOT EXISTS main.stocks.nvda_raw")


Store the yFinance Data in UC

In [0]:
raw_path = "/Volumes/main/stocks/nvda_raw"  # folder for Bronze source files

#data = data.reset_index()  # make Date a normal column
spark_df = spark.createDataFrame(data)

(
    spark_df
    .write
    .mode("overwrite")   
    .option("header", True)
    .csv(raw_path)
)


In [0]:
display(
    spark.read.format("csv")
    .option("header", "true")
    .load(raw_path)
    .limit(5)
)
